# Use OpenAI SDK with Phi-3 in Azure AI and Azure ML

Use `openai` SDK to consume Phi-3 deployments in Azure AI and Azure ML. The Phi-3 family of models in Azure AI and Azure ML offers an API compatible with the OpenAI Chat Completion API. It allows customers and users to transition seamlessly from OpenAI models to Phi-3 LLMs. 

The API can be directly used with OpenAI's client libraries or third-party tools, like LangChain or LlamaIndex.

The example below shows how to make this transition using the OpenAI Python Library. Notice that Phi-3 supports only chat completions API.

> Review the [documentation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-phi3) for the Phi-3 family of models at for AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home) or Azure Machine Learning.
* Create a project or workspace.
* Go to the Model Catalog and search for any of the models of the Phi-3 family. This example shows a `phi-3-mini-128k-instruct` (chat completions) and a `phi-3-vision-128k-instruct` (chat completions with vision).
* Deploy it using either "Serverless API endpoints" or "Self-hosted Online Endpoints".

    > Notice that `phi-3-vision-128k-instruct` can only be deployed to Self-hosted Online Endpoints. You need to ensure you have enough quota in yur subscription to use this model. You can always use our temporary quota access to have an endpoint working for 7 days.

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-phi-3) for model deployment and inference.

To complete this tutorial, you will need to:

* Install `openai`:

    ```bash
    pip install openai
    ```

## Example

The Phi-3-Mini-128K-Instruct is a 3.8 billion-parameter, lightweight, state-of-the-art open model trained using the Phi-3 datasets. This dataset includes both synthetic data and filtered publicly available website data, with an emphasis on high-quality and reasoning-dense properties. The model belongs to the Phi-3 family with the Mini version in two variants 4K and 128K which is the context length (in tokens) that it can support.

The following is an example about how to use `openai` with a `phi-3-mini-128k-instruct` model deployed in Azure AI and Azure ML. 

In [ ]:
from openai import OpenAI

You will need to have a Endpoint url and Authentication Key associated with that endpoint. This can be acquired from previous steps. 
To work with `openai`, configure the client as follows:

- `base_url`: Use the endpoint URL from your deployment. Include `/v1` as part of the URL.
- `api_key`: Use your API key.

In [ ]:
client = OpenAI(base_url="<endpoint-url>/v1", api_key="<key>")

Use the client to create chat completions requests:

In [ ]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Who is the most renowned French painter? Provide a short answer.",
        }
    ],
    model="azureai",
)

The generated text can be accessed as follows:

In [ ]:
print(response.choices[0].message.content)

Limitations:
- Only `user` and `assistant` roles are supported by Phi-3. Using the role `system` will get the model to ignore those messages.
- Tools is not supported.
- Output type `json_object` is not supported. You can still prompt the model to generate JSON responses, but you have to validate that the response is a valid JSON object.

## Phi-3 Vision

Phi-3 Vision is a lightweight, state-of-the-art open multimodal model built upon datasets which include - synthetic data and filtered publicly available websites - with a focus on very high-quality, reasoning dense data both on text and vision. The model belongs to the Phi-3 model family, and the multimodal version comes with 128K context length (in tokens) it can support. The model underwent a rigorous enhancement process, incorporating both supervised fine-tuning and direct preference optimization to ensure precise instruction adherence and robust safety measures. 

Let's create a client to connect to the model. Use the endpoint URL and append `/v1` to it:

In [ ]:
client = OpenAI(base_url="<endpoint-url>/v1", api_key="<key>")

In [ ]:
from urllib.request import urlopen, Request
import os
import base64

image_url = "https://news.microsoft.com/source/wp-content/uploads/2024/04/The-Phi-3-small-language-models-with-big-potential-1-1900x1069.jpg"
image_format = "jpeg"

request = Request(image_url, headers={"User-Agent": "Mozilla/5.0"})
image_data = base64.b64encode(urlopen(request).read()).decode("utf-8")
data_url = f"data:image/{image_format};base64,{image_data}"

You can visualize the image:

In [ ]:
import requests
import IPython.display as Disp

Disp.Image(requests.get(image_url).content)

Let's see what Phi-3 can analyze about it:

In [ ]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Which conclusion can be extracted from the following chart?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": data_url,
                    },
                },
            ],
        }
    ],
    model="azureai",
    max_tokens=2048,
)

print(response.choices[0].message.content)

## Aditional resources

Here are some additional reference:  

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)
* [Phi-3 family of models](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-phi3)